In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [2]:
df1 = pd.read_csv('../data/features_basicas.csv', low_memory=False, index_col='person')
df2 = pd.read_csv('../data/features_checkouts.csv', low_memory=False, index_col='person')
df3 = pd.read_csv('../data/features_vistos.csv', low_memory=False, index_col='person')
df4 = pd.read_csv('../data/feature_dispositivo_mas_usado.csv', low_memory=False, index_col='person')
df5 = pd.read_csv('../data/feature_dia_mas_activo.csv', low_memory=False, index_col='person')
df6 = pd.read_csv('../data/feature_compra_onsite.csv', low_memory=False, index_col='person')
df7 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos.csv', low_memory=False, index_col='person')
df8 = pd.read_csv('../data/feature_dispositivos_storage.csv', low_memory=False, index_col='person')
df9 = pd.read_csv('../data/featureUsuarioSonDeSaoPablo.csv', low_memory=False, index_col='person')
df10 = pd.read_csv('../data/featureCantidadMaximaPersonaVeUnProducto.csv', low_memory=False, index_col='person')
df11 = pd.read_csv('../data/featureUltimaConexion.csv', low_memory=False, index_col='person')
df12 = pd.read_csv('../data/features_basicas_ult_quin.csv', low_memory=False, index_col='person')
df14 = pd.read_csv('../data/days_elapsed_from_last_event.csv', low_memory=False, index_col='person')
df15 = pd.read_csv('../data/featureUsuarioRealiza30EventosEn20MinsLosUltimos2Dias.csv', low_memory=False, index_col='person')
df16 = pd.read_csv('../data/features_ultimo_checkout.csv', low_memory=False, index_col='person')
df17 = pd.read_csv('../data/diferencia_de_count_de_eventos.csv', low_memory=False, index_col='person')
df18 = pd.read_csv('../data/top_10_celulares.csv', low_memory=False, index_col='person')
df19 = pd.read_csv('../data/feature_cantidad_de_eventos_en_intervalos_ult_quin.csv', low_memory=False, index_col='person')

labels = pd.read_csv('../data/labels_training_set.csv', low_memory=False, index_col='person')

df_unidos = df1.join(df2, how='inner')
df_unidos = df_unidos.join(df3, how='inner')
df_unidos = df_unidos.join(df4, how='inner')
df_unidos = df_unidos.join(df5, how='inner')
df_unidos = df_unidos.join(df6, how='inner')
df_unidos = df_unidos.join(df7, how='inner')
df_unidos = df_unidos.join(df8, how='inner')
df_unidos = df_unidos.join(df9, how='inner')
df_unidos = df_unidos.join(df10, how='inner')
df_unidos = df_unidos.join(df11, how='inner')
df_unidos = df_unidos.join(df12, how='inner')
df_unidos = df_unidos.join(df14, how='inner')
df_unidos = df_unidos.join(df15, how='inner')
df_unidos = df_unidos.join(df16, how='inner')
df_unidos = df_unidos.join(df17).fillna(0)
df_unidos = df_unidos.join(df18, how='inner')
df_unidos = df_unidos.join(df19, how='inner')

#  Le asigno las features al subset de usuarios que nos da Trocafone para entrenar.
df_test = df_unidos.join(labels, on='person', how='inner')
df_ensamble = df_unidos.reset_index()[['person']].set_index('person')
df_test.shape

(19414, 93)

In [3]:
df_unidos.shape

(38829, 92)

In [4]:
df_ensamble.shape

(38829, 0)

In [5]:
features = df_test.columns.tolist()
features.remove('label')

In [6]:
#  Partimos los datos que tenemos para entrenar en dos partes, una para entrenar el modelo (80% de los datos)
#  y la otra parte se usará para probar el modelo (el 20% restante)
X_train, X_test, Y_train, Y_test = train_test_split(df_test[features],\
                                                    df_test['label'], test_size=0.20, random_state=80)

## RandomForest

In [7]:
rf = RandomForestClassifier(n_estimators=3000, n_jobs=-1, min_samples_split=300,\
                            max_depth=7, random_state=80, class_weight='balanced')
rf.fit(X_train,Y_train)
Y_pred_rf = rf.predict_proba(df_unidos)
feature_rf = pd.Series([x[1] for x in Y_pred_rf])
df_ensamble['pred_rf'] = feature_rf.values

## LightGBM_v1

In [8]:
model_lgbm_v1 = lgb.LGBMClassifier(random_state=80, num_leaves=1000,\
                               n_estimators=500, learning_rate=0.01, max_depth=5)
model_lgbm_v1.fit(X_train, Y_train)
Y_pred_lgbm_v1 = model_lgbm_v1.predict_proba(df_unidos)
feature_lgbm_v1 = pd.Series([x[1] for x in Y_pred_lgbm_v1])
df_ensamble['pred_lgbm_v1'] = feature_lgbm_v1.values

## LightGBM_v2

In [9]:
model_lgbm_v2 = lgb.LGBMClassifier(random_state=80,\
                               n_estimators=1000, learning_rate=0.01, max_depth=4, gamma=1)
model_lgbm_v2.fit(X_train, Y_train)
Y_pred_lgbm_v2 = model_lgbm_v2.predict_proba(df_unidos)
feature_lgbm_v2 = pd.Series([x[1] for x in Y_pred_lgbm_v2])
df_ensamble['pred_lgbm_v2'] = feature_lgbm_v2.values

## XGBoost_v2

In [10]:
model_xgb_v2 = XGBClassifier(scale_pos_weight=1, n_estimators=100, random_state=80, max_depth=3)
model_xgb_v2.fit(X_train, Y_train)
Y_pred_xgb_v2 = model_xgb_v2.predict_proba(df_unidos)
feature_xgb_v2 = pd.Series([x[1] for x in Y_pred_xgb_v2])
df_ensamble['pred_xgb_v2'] = feature_xgb_v2.values

In [11]:
df_ensamble = df_ensamble.join(df_unidos, how='inner')

In [12]:
df_ensamble.shape

(38829, 96)

## Ensamble de los 4 modelos usando XGBoost_v2

In [13]:
df_test_ensamble = df_ensamble.join(labels, on='person', how='inner')
features_ensamble = df_test_ensamble.columns.tolist()
features_ensamble.remove('label')
X_train_ens, X_test_ens, Y_train_ens, Y_test_ens = train_test_split(df_test_ensamble[features_ensamble],\
                                                    df_test_ensamble['label'], test_size=0.20, random_state=80)

In [14]:
ensamble = RandomForestClassifier(n_estimators=3000, n_jobs=-1, min_samples_split=300,\
                            max_depth=7, random_state=80, class_weight='balanced')
ensamble.fit(X_train_ens, Y_train_ens)
y_pred = ensamble.predict_proba(X_test_ens)
y_pred_proba = [p[1] for p in y_pred]
print(roc_auc_score(Y_test_ens, y_pred_proba))

0.8603099414681029


In [15]:
#  Acá quedan ordenadas las features por importancia para el modelo, aparentemente la marca más vista y la marca
#  que más compró cada usuario no es tan importante.
feature_importances = pd.DataFrame(ensamble.feature_importances_,\
                                   index = df_test_ensamble[features_ensamble].columns,\
                                    columns=['importance']).sort_values('importance',ascending=False)

feature_importances.head(5)

,importance
pred_lgbm_v1,0.222612
pred_lgbm_v2,0.179526
pred_xgb_v2,0.120616
pred_rf,0.115302
storage_mas_checkout,0.049296


In [16]:
df_submit = pd.read_csv('../data/trocafone_kaggle_test.csv', low_memory=False, index_col='person')
df_events = df_submit.join(df_ensamble, how='inner')

In [17]:
df_events.shape

(19415, 96)

In [18]:
kaggle_pred = ensamble.predict_proba(df_events)
proba_de_comprar = [x[1] for x in kaggle_pred]
series = pd.Series(proba_de_comprar)
df_submit['label'] = series.values

In [19]:
df_submit.to_csv('../submit_ensamble.csv')

In [20]:
df_ensamble.head()

,pred_rf,pred_lgbm_v1,pred_lgbm_v2,pred_xgb_v2,tiene_checkouts,llegaron_por_ad,llegaron_por_search,tiene_conversions,cant_conversions,cant_checkouts,...,diff_checkouts_15_dias,diff_visited_site_15_dias,diff_searched_products_15_dias,diff_viewed_products_15_dias,total_top_10_phones_checkout,different_top_10_phones_checkout,ult_quin_cant_eventos_4_a_10hs,ult_quin_cant_eventos_11_a_14hs,ult_quin_cant_eventos_15_a_20hs,ult_quin_cant_eventos_21_a_3hs
person,,,,,,,,,,,,,,,,,,,,,
4886f805,0.047171,0.003316,0.003194,0.004160,True,False,True,False,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0
ad93850f,0.362707,0.091421,0.075257,0.087732,True,True,True,False,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,65.0
0297fc1e,0.377176,0.009888,0.009313,0.017897,True,True,False,False,0.0,7.0,...,0.0,21.0,0.0,73.0,1.0,1.0,3.0,31.0,74.0,27.0
2d681dd8,0.097846,0.010542,0.009956,0.008807,True,True,True,False,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,16.0,10.0
cccea85e,0.681406,0.122966,0.045670,0.107495,True,True,True,False,0.0,1.0,...,-1.0,12.0,1.0,311.0,0.0,0.0,0.0,77.0,215.0,297.0
